# [LVV-T2213] LUT Application from MTMount Elevation Changes - Data Analysis

This notebook contains plots to analyze the data obtained when running the [LVV-T2213] test case.  
It is based on the notebooks originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.   

Please, see the [README] file for the requirements to run this notebook.  

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2213]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2213
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

In [ ]:
test_case = "LVV-T2213"
test_exec = "LVV-E0000"

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from astropy.time import Time
from matplotlib import pyplot as plt

from lsst.sitcom import vandv

In [ ]:
client = vandv.efd.create_efd_client()

exec_info = vandv.ExecutionInfo()
print(exec_info)

## Check log from the EFD

Use the code below to query the data from the EFD.  
Remember that you can use the index to help selecting the data.  

In [ ]:
date = "2022-08-30"

log_df = await client.select_time_series(
    "lsst.sal.Script.logevent_logMessage", 
    f"message",
    Time(f"{date}T00:00:00", format="isot", scale="utc"),
    Time(f"{date}T23:59:00", format="isot", scale="utc"),
    index=-22130830
)

log_df[log_df.message.str.contains(test_exec)]

In [ ]:
t_start = Time(log_df[log_df.message.str.contains("START")].index[1], scale="utc")
t_end = Time(log_df[log_df.message.str.contains("END")].index[-1], scale="utc")

print(f"Actual test happened between\n {t_start}\n and\n {t_end}\n")

## Elevation LUT testing 

We want to see how the different components behave during the Elevation LUT testing.  
For this, we want to select the associated data between the START/END.  
If the test is run more than one time, you might need to modify the cells below accordingly.

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "START", "Elevation LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-08-30 00:00", scale="utc", format="iso"),
    num=20)

t_start = vandv.efd.time_pd_to_astropy(df.index[0].floor("1s"))
print(t_start, df.iloc[0]["message"])

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "END", "Elevation LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-08-30 00:00", scale="utc", format="iso"),
    num=20)

t_end = vandv.efd.time_pd_to_astropy(df.index[0].ceil("1s"))
print(t_end, df.iloc[0]["message"])

### Analyse M1M3 Data

In [ ]:
fad = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start,
    t_end)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    "*", 
    t_start,
    t_end)

fig, axs = plt.subplot_mosaic(
    mosaic="A",
    num="Slew Without Correction", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 4),
)

fig.suptitle("M1M3 Timeline - Slew LUT+inclinometer")
_ = vandv.m1m3.timeline_zforces(axs["A"], fad, "zForce", elevation=el)

#### Snapshot at 86 deg.

In [ ]:
title = "M1M3 Snapshot - LUT+inclinometer at 86 deg - XYZ end slew"

el86 = el[el.actualPosition == 86.]
t_start0 = Time(el86.index[0])
t_end0 = Time(el86.index[-1])

fel86 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    "*", 
    t_start0,
    t_end0)

fba86 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
    "*",
    t_start0,
    t_end0)

fst86 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedStaticForces",
    "*",
    t_start0,
    t_end0)

fao86 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
    "*", 
    t_start0,
    t_end0)

fad86 = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start0,
    t_end0)

series = [df.iloc[-1] for df in [fel86, fba86, fst86, fao86, fad86]]
labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "appliedActiveOpticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num=title, 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

fig.suptitle(title)
_ = vandv.m1m3.snapshot_xforces(axs["A"], series)
_ = vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
_ = vandv.m1m3.snapshot_zforces(axs["C"], series)

#### Snapshot at 82.5 deg

In [ ]:
title = "M1M3 Snapshot - LUT+inclinometer at 82.5 deg - XYZ end slew"

el82 = el[el.actualPosition == 82.5]
t_start1 = Time(el82.index[0])
t_end1 = Time(el82.index[-1])

fel82 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    "*", 
    t_start1,
    t_end1)

fba82 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
    "*",
    t_start1,
    t_end1)

fst82 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedStaticForces",
    "*",
    t_start1,
    t_end1)

fao82 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
    "*", 
    t_start1,
    t_end1)

fad82 = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start1,
    t_end1)

series = [df.iloc[-1] for df in [fel82, fba82, fst82, fao82, fad82]]
labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "appliedActiveOpticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num=title, 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

fig.suptitle(title)
_ = vandv.m1m3.snapshot_xforces(axs["A"], series)
_ = vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
_ = vandv.m1m3.snapshot_zforces(axs["C"], series)

#### Snapshot differences

In [ ]:
title = "M1M3 Snapshot - LUT+inclinometer difference between 86 deg and 82.5 deg - XYZ end slew"

series = [df86.select_dtypes(['number']).iloc[-1] - df82.select_dtypes(['number']).iloc[-1] 
          for df82, df86 in zip(
              [fel82, fba82, fst82, fao82, fad82], 
              [fel86, fba86, fst86, fao86, fad86]
          )]

labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "appliedActiveOpticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num=title, 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

fig.suptitle(title)
_ = vandv.m1m3.snapshot_xforces(axs["A"], series)
_ = vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
_ = vandv.m1m3.snapshot_zforces(axs["C"], series)

### Analyse M2 Data

In [ ]:
axf = await client.select_time_series(
    "lsst.sal.MTM2.axialForce", 
    "*",
    t_start,
    t_end)

taf = await client.select_time_series(
    "lsst.sal.MTM2.tangentForce", 
    "*",
    t_start,
    t_end)

cof = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*", 
    t_start,
    t_end)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC\nD",
    num="Slew Without Correction - M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 8),
)

cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
]

fig.suptitle("M2 Timeline - Elevation Test\n LUT+inclinometer - Per Actuator")
_ = vandv.m2.timeline_axial_forces_per_act(axs["A"], axf, elevation=el, act="B1", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["B"], axf, elevation=el, act="B8", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["C"], axf, elevation=el, act="B16", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["D"], axf, elevation=el, act="B24", cols=cols)

### Analyse Camhex Data

In [ ]:
# From the XML:
#   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
#   Linear positions are in microns, angular positions are in degrees.
pos = await client.select_time_series(
    "lsst.sal.MTHexapod.application",
    "*",
    t_start,
    t_end,
    index=1
)

# Unravel in x/y/z/u/v/w
for i, col in enumerate("xyzuvw"):
    pos[col] = pos[f"position{i}"]


# Triggered at the end of a slew
cpos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_compensatedPosition",
    "*",
    t_start,
    t_end,
    index=1
)

# Triggered only after move/offset. Should not see much. 
upos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
    "*",
    t_start,
    t_end,
    index=1
)

# Estimate the LUT position
lut_pred = vandv.hexapod.get_lut_positions(index=1, elevation=el.actualPosition)
lut = pd.DataFrame(lut_pred, columns=["x", "y", "z", "u", "v", "w"], index=el.index)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="Elevation Test\n CamHex Actual Position", 
    dpi=120,
    figsize=(10, 6),
    sharex=True,
)

cols = "xyzuvw"
for ax, col in zip("ABCDEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos, lut], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s", ""],
        names=["Actual Position", "Compensated", "Uncompensated", "LUT"]
    )

# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.5, -1), ncol=5)

fig.suptitle("CamHex Timeline - Slew w/out Compensation")
fig.autofmt_xdate()
fig.tight_layout()

### Analyse M2Hex Data

In [ ]:
# From the XML:
#   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
#   Linear positions are in microns, angular positions are in degrees.
pos = await client.select_time_series(
    "lsst.sal.MTHexapod.application",
    "*",
    t_start,
    t_end,
    index=2
)

# Unravel in x/y/z/u/v/w
for i, col in enumerate("xyzuvw"):
    pos[col] = pos[f"position{i}"]


# Triggered at the end of a slew
cpos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_compensatedPosition",
    "*",
    t_start,
    t_end,
    index=2
)

# Triggered only after move/offset. Should not see much. 
upos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
    "*",
    t_start,
    t_end,
    index=2
)

# Estimate the LUT position
lut_pred = vandv.hexapod.get_lut_positions(index=2, elevation=el.actualPosition)
lut = pd.DataFrame(lut_pred, columns=["x", "y", "z", "u", "v", "w"], index=el.index)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="Slew Without LUT Correction - M2Hex Actual Position", 
    dpi=120,
    figsize=(10, 6),
    sharex=True,
)

cols = "xyzuvw"
for ax, col in zip("ABCDEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos, lut], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s", ""],
        names=["Actual Position", "Compensated", "Uncompensated", "LUT"]
    )

# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.5, -1), ncol=5)

fig.suptitle("M2Hex Timeline - Slew w/out Compensation")
fig.autofmt_xdate()
fig.tight_layout()

## Azimuth LUT testing

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "START", "Azimuth LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-08-30 00:00", scale="utc", format="iso"),
    num=20)

t_start = vandv.efd.time_pd_to_astropy(df.index[0].floor("1s"))
print(t_start, df.iloc[0]["message"])

In [ ]:
df = await vandv.efd.query_script_message_contains(
    client=client, 
    contains=[test_case, test_exec, "END", "Azimuth LUT"],
    upper_t=Time.now(),
    lower_t=Time("2022-08-30 00:00", scale="utc", format="iso"),
    num=20)

t_end = vandv.efd.time_pd_to_astropy(df.index[0].ceil("1s"))
print(t_end, df.iloc[0]["message"])

### Analyse M1M3 Data

In [ ]:
fad = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start,
    t_end)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    "*", 
    t_start,
    t_end)

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    "*", 
    t_start,
    t_end)

fig, axs = plt.subplot_mosaic(
    mosaic="A",
    num="Slew Without Correction", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 4),
)

fig.suptitle("M1M3 Timeline - Slew LUT+inclinometer")
_ = vandv.m1m3.timeline_zforces(axs["A"], fad, "zForce", elevation=el)

#### Snapshot at Az 90 deg.

In [ ]:
title = "M1M3 Snapshot - LUT+inclinometer at 84 deg - XYZ end slew"

el84 = el[el.actualPosition == 84.]
t_start0 = Time(el84.index[0])
t_end0 = Time(el84.index[-1])

fel84 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    "*", 
    t_start0,
    t_end0)

fba84 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
    "*",
    t_start0,
    t_end0)

fst84 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedStaticForces",
    "*",
    t_start0,
    t_end0)

fao84 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
    "*", 
    t_start0,
    t_end0)

fad84 = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start0,
    t_end0)

series = [df.iloc[-1] for df in [fel84, fba84, fst84, fao84, fad84]]
labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "appliedActiveOpticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num=title, 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

fig.suptitle(title)
_ = vandv.m1m3.snapshot_xforces(axs["A"], series)
_ = vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
_ = vandv.m1m3.snapshot_zforces(axs["C"], series)

#### Snapshot at 85 deg

In [ ]:
title = "M1M3 Snapshot - LUT+inclinometer at 85 deg - XYZ end slew"

el85 = el[el.actualPosition == 85]
t_start1 = Time(el85.index[0])
t_end1 = Time(el85.index[-1])

fel85 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedElevationForces", 
    "*", 
    t_start1,
    t_end1)

fba85 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedBalanceForces", 
    "*",
    t_start1,
    t_end1)

fst85 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedStaticForces",
    "*",
    t_start1,
    t_end1)

fao85 = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_appliedActiveOpticForces",
    "*", 
    t_start1,
    t_end1)

fad85 = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    "*", 
    t_start1,
    t_end1)

series = [df.iloc[-1] for df in [fel85, fba85, fst85, fao85, fad85]]
labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "appliedActiveOpticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num=title, 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

fig.suptitle(title)
_ = vandv.m1m3.snapshot_xforces(axs["A"], series)
_ = vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
_ = vandv.m1m3.snapshot_zforces(axs["C"], series)

#### Snapshot differences

In [ ]:
title = "M1M3 Snapshot - LUT+inclinometer difference between 84 deg and 85 deg - XYZ end slew"

series = [df85.select_dtypes(['number']).iloc[-1] - df84.select_dtypes(['number']).iloc[-1] 
          for df85, df84 in zip(
              [fel85, fba85, fst85, fao85, fad85], 
              [fel84, fba84, fst84, fao84, fad84]
          )]

labels = [
    "appliedElevationForces", 
    "appliedBalanceForces", 
    "appliedStaticForces",
    "appliedActiveOpticForces",
    "forceActuatorData",
]

fig, axs = plt.subplot_mosaic(
    mosaic="AB\nAB\nCC\nCC",
    num=title, 
    constrained_layout=True,
    dpi=120,
    figsize=(9, 5),
)

fig.suptitle(title)
_ = vandv.m1m3.snapshot_xforces(axs["A"], series)
_ = vandv.m1m3.snapshot_yforces(axs["B"], series, labels=labels)
_ = vandv.m1m3.snapshot_zforces(axs["C"], series)

### Analyse M2 Data

In [ ]:
axf = await client.select_time_series(
    "lsst.sal.MTM2.axialForce", 
    "*",
    t_start,
    t_end)

taf = await client.select_time_series(
    "lsst.sal.MTM2.tangentForce", 
    "*",
    t_start,
    t_end)

cof = await client.select_time_series(
    "lsst.sal.MTM2.command_applyForces",
    "*", 
    t_start,
    t_end)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="A\nB\nC\nD",
    num="Slew Without Correction - M2", 
    constrained_layout=True,
    dpi=120,
    figsize=(12, 8),
)

cols = [
    # "applied", 
    # "hardpointCorrection", 
    "lutGravity", 
    # "lutTemperature", 
    "measured"
]

fig.suptitle("M2 Timeline - Elevation Test\n LUT+inclinometer - Per Actuator")
_ = vandv.m2.timeline_axial_forces_per_act(axs["A"], axf, elevation=el, act="B1", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["B"], axf, elevation=el, act="B8", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["C"], axf, elevation=el, act="B16", cols=cols)
_ = vandv.m2.timeline_axial_forces_per_act(axs["D"], axf, elevation=el, act="B24", cols=cols)

### Analyse Camhex Data

In [ ]:
# From the XML:
#   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
#   Linear positions are in microns, angular positions are in degrees.
pos = await client.select_time_series(
    "lsst.sal.MTHexapod.application",
    "*",
    t_start,
    t_end,
    index=1
)

# Unravel in x/y/z/u/v/w
for i, col in enumerate("xyzuvw"):
    pos[col] = pos[f"position{i}"]


# Triggered at the end of a slew
cpos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_compensatedPosition",
    "*",
    t_start,
    t_end,
    index=1
)

# Triggered only after move/offset. Should not see much. 
upos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
    "*",
    t_start,
    t_end,
    index=1
)

# Estimate the LUT position
lut_pred = vandv.hexapod.get_lut_positions(index=1, elevation=el.actualPosition)
lut = pd.DataFrame(lut_pred, columns=["x", "y", "z", "u", "v", "w"], index=el.index)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="Elevation Test\n CamHex Actual Position", 
    dpi=120,
    figsize=(10, 6),
    sharex=True,
)

cols = "xyzuvw"
for ax, col in zip("ABCDEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos, lut], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s", ""],
        names=["Actual Position", "Compensated", "Uncompensated", "LUT"]
    )

# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.5, -1), ncol=5)

fig.suptitle("CamHex Timeline - Slew w/out Compensation")
fig.autofmt_xdate()
fig.tight_layout()

### Analyse M2Hex Data

In [ ]:
# From the XML:
#   Actual MTHexapod position, in order (X, Y, Z, U, V, W). 
#   Linear positions are in microns, angular positions are in degrees.
pos = await client.select_time_series(
    "lsst.sal.MTHexapod.application",
    "*",
    t_start,
    t_end,
    index=2
)

# Unravel in x/y/z/u/v/w
for i, col in enumerate("xyzuvw"):
    pos[col] = pos[f"position{i}"]


# Triggered at the end of a slew
cpos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_compensatedPosition",
    "*",
    t_start,
    t_end,
    index=2
)

# Triggered only after move/offset. Should not see much. 
upos = await client.select_time_series(
    "lsst.sal.MTHexapod.logevent_uncompensatedPosition",
    "*",
    t_start,
    t_end,
    index=2
)

# Estimate the LUT position
lut_pred = vandv.hexapod.get_lut_positions(index=2, elevation=el.actualPosition)
lut = pd.DataFrame(lut_pred, columns=["x", "y", "z", "u", "v", "w"], index=el.index)

In [ ]:
fig, axs = plt.subplot_mosaic(
    mosaic="AD\nBE\nCF",
    num="Slew Without LUT Correction - M2Hex Actual Position", 
    dpi=120,
    figsize=(10, 6),
    sharex=True,
)

cols = "xyzuvw"
for ax, col in zip("ABCDEF", cols):
    _ = vandv.hexapod.timeline_position(
        axs[ax], 
        [pos, cpos, upos, lut], 
        column=col, 
        elevation=el, 
        symbols=["", "o", "s", ""],
        names=["Actual Position", "Compensated", "Uncompensated", "LUT"]
    )

# Hide xlabel
for i in "ABDE":
    _ = axs[i].set_xlabel("")
    
_ = axs["F"].legend(loc='lower center', bbox_to_anchor=(0.5, -1), ncol=5)

fig.suptitle("M2Hex Timeline - Slew w/out Compensation")
fig.autofmt_xdate()
fig.tight_layout()